In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import graph as gr
import utils as ut
import feature_selection as fs
import classification as cl
import os.path
import matplotlib.pyplot as plt
from pprint import pprint
import importlib
import community
import seaborn as sns
from gensim.models import KeyedVectors
from pyemd import emd
from gensim import corpora
from nltk.corpus import stopwords
from gensim.matutils import softcossim
import copy
import itertools
import pickle
import PathSim as ps
from math import sqrt
import time as tm

Using Theano backend.


In [10]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense, Input
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Dropout, Activation, Reshape
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, roc_auc_score
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt

In [68]:
#########dataset and domain selection : sleep apnea: 0 ; obesity:1
importlib.reload(ut)
select_domain  = 1
domain = ['apnea','obesity']
root = '../linkPrediction/'
filepath=[['dataset/apnea-all,3.csv',
         'dataset/apnea-distinct_keyword.csv'],
        ['dataset/obesity-all,3.csv',
         'dataset/obesity-distinct_keyword.csv']]
########path to save graph files
graphpath=root+'graphs/'+domain[select_domain]
########path to save result files, data files and resultent figures
# datapath=root+'dataframes/'+domain[select_domain]
multipath=root+'multi_edges/'+domain[select_domain]
datapath=multipath
#path to save classification models
modelpath=root+'models/'+domain[select_domain]
########columns with sub-columns
column_split=['keyword','author_name','affiliation_1','affiliation_2','country']
#time=[parent_year, train:start_year, test:start_year, no_of_test_years(1), total_year_in_each_iteration, no of grandparents]
times=[2007,2008,2015,1,1,20]
######load dataset and keyword list into dataframes
df, key_list = ut.load_dataset(filepath[select_domain], column_split)

In [48]:
# time=[1991,1992,1994,1,1,20]
importlib.reload(gr)
g_train, g_test, g_parent, g_train_static, g_test_static = gr.graph_load(graphpath, times)
# g_train, g_test = gr.graph_norm(root, domain, select_domain, g_train, g_test)

../linkPrediction/graphs/apnea\train_graph_2008.gpickle loaded, nodes: 459 edges: 773
../linkPrediction/graphs/apnea\train_graph_2009.gpickle loaded, nodes: 470 edges: 688
../linkPrediction/graphs/apnea\train_graph_2010.gpickle loaded, nodes: 496 edges: 786
../linkPrediction/graphs/apnea\train_graph_2011.gpickle loaded, nodes: 536 edges: 977
../linkPrediction/graphs/apnea\train_graph_2012.gpickle loaded, nodes: 584 edges: 1054
../linkPrediction/graphs/apnea\train_graph_2013.gpickle loaded, nodes: 634 edges: 1280
../linkPrediction/graphs/apnea\train_graph_2014.gpickle loaded, nodes: 583 edges: 1144
../linkPrediction/graphs/apnea\test_graph_2015.gpickle loaded, nodes: 1093 edges: 2019
../linkPrediction/graphs/apnea\parent_graph_2007-2008.gpickle loaded, nodes: 388 edges: 689
../linkPrediction/graphs/apnea\train_graph_2008-2015.gpickle loaded, nodes: 1093 edges: 5142
../linkPrediction/graphs/apnea\test_graph_2015-2016.gpickle loaded, nodes: 1093 edges: 2019


In [49]:
s_key={0:[9875,1020],1:[7614,9953]}
for t in range(2008,2015):
    g_train[t].remove_nodes_from(s_key[select_domain])
g_test[2015].remove_nodes_from(s_key[select_domain])
g_parent.remove_nodes_from(s_key[select_domain])
g_train_static.remove_nodes_from(s_key[select_domain])
g_test_static.remove_nodes_from(s_key[select_domain])

In [50]:
importlib.reload(gr)
assoc=0
g_train, g_test, g_parent, g_train_static, g_test_static = gr.graph_load(graphpath, times)
g_train, g_test = gr.graph_norm(root, domain, select_domain, g_train, g_test,0.0001)

# for t in range(2008,2015):
#     ts_nodes = set(g_test[2015].nodes())
#     tr_nodes = set(g_train[t].nodes())
#     dif_nodes = tr_nodes.difference(ts_nodes)
#     g_train[t].remove_nodes_from(dif_nodes)
#     print(len(tr_nodes),len(dif_nodes), len(g_train[t].nodes),len(g_train[t].edges))

../linkPrediction/graphs/apnea\train_graph_2008.gpickle loaded, nodes: 459 edges: 773
../linkPrediction/graphs/apnea\train_graph_2009.gpickle loaded, nodes: 470 edges: 688
../linkPrediction/graphs/apnea\train_graph_2010.gpickle loaded, nodes: 496 edges: 786
../linkPrediction/graphs/apnea\train_graph_2011.gpickle loaded, nodes: 536 edges: 977
../linkPrediction/graphs/apnea\train_graph_2012.gpickle loaded, nodes: 584 edges: 1054
../linkPrediction/graphs/apnea\train_graph_2013.gpickle loaded, nodes: 634 edges: 1280
../linkPrediction/graphs/apnea\train_graph_2014.gpickle loaded, nodes: 583 edges: 1144
../linkPrediction/graphs/apnea\test_graph_2015.gpickle loaded, nodes: 1093 edges: 2019
../linkPrediction/graphs/apnea\parent_graph_2007-2008.gpickle loaded, nodes: 388 edges: 689
../linkPrediction/graphs/apnea\train_graph_2008-2015.gpickle loaded, nodes: 1093 edges: 5142
../linkPrediction/graphs/apnea\test_graph_2015-2016.gpickle loaded, nodes: 1093 edges: 2019
../linkPrediction/Informetrics/

In [51]:
importlib.reload(ps)

def get_lists(g):    
    node_article = []
    art_author = []
    node_list=[]
    aut_list=[]
    art_list=[]
    for nd, row in g.nodes(data=True):
        node_list.append(nd)
        for art_s in row['art_id']:
            art_list.append(art_s)
            node_article.append((nd,art_s))
    for art_s in art_list:
        a = df.loc[df['art_id']==art_s,['art_year','author_name']]
        aa = a['author_name'].values[0]
        a = list(itertools.product([art_s], aa))
        art_author.extend(a)
        aut_list.extend(aa)
    node_list=list(set(node_list))
    aut_list=list(set(aut_list))
    art_list=list(set(art_list))
    return node_list, aut_list, art_list, node_article, art_author


def creat_adj(node_list, aut_list, art_list, node_article, art_author):
    #  Create the node-aut adjacency

    node_art_df = pd.DataFrame(node_article, columns=['node', 'article'])
    node_art_adj = pd.DataFrame(0, index=node_list, columns=art_list)
    for nd, art in zip(node_art_df['node'], node_art_df['article']):
        node_art_adj.at[nd, art]=1
    node_art_adj = node_art_adj.dropna(axis='columns')

    # #  Create the aut-art adjacency
    #
    art_aut_df = pd.DataFrame(art_author, columns=['article','author'])
    art_aut_adj = pd.DataFrame(0, index=art_list, columns=aut_list)
    for art, aut in zip(art_aut_df['article'], art_aut_df['author']):
        art_aut_adj.at[art, aut]=1
    art_aut_adj = art_aut_adj.dropna(axis='columns')
    return node_art_adj,art_aut_adj

def calculate_pathsim(g,f_node):
    node_list, aut_list, art_list, node_article, art_author = get_lists(g)
    mult = lambda x: np.asarray(x) * np.asarray(s)
    node_art_adj,art_aut_adj=creat_adj(node_list, aut_list, art_list, node_article, art_author)
    result = ps.commute_matrix(node_art_adj, art_aut_adj, f_node)
#     AD, s = ps.scaling(result, f_node)
# #     an_hai = AD.apply(mult)
#     AD = AD.sort_values('pathsim_score', ascending=False)
    return result

In [52]:
importlib.reload(fs)
importlib.reload(cl)
#ration of negative:positive train instance:(eg,10 for 10:1=neg:pos)
freq=10
##### time=[2001,2007,2015,1,1,10]
##### train_data building: dictionary of each years non-connected node pairs dataframes
train_data, train_data_static, parent_data, test_data_static, edge_list = cl.non_edge_feature_dataframe(g_train, 
                                                                                                         g_test,
                                                                                                         g_parent,
                                                                                                         g_train_static, 
                                                                                                         g_test[2015],  
                                                                                                         times, 
                                                                                                         freq)
### train_data: non-connected node pairs with feature values
###node_features: node feature values
node_feature, train_data = fs.dynamic_graph_feature_set(df, key_list, train_data, g_parent, g_train, g_train_static, times)

importlib.reload(ps)
for t in range(2008,2015):
    if t==2015:
        g=g_test[t]
    else:
        g=g_train[t]
    f_node = list(g.nodes)[0]
    result = calculate_pathsim(g,f_node)
    train_data[t]['path_sim'] = train_data[t].apply(lambda row: result[row['row_name'][0]][row['row_name'][1]],axis=1)

pos
test_size: 682
neg
test_size: 590954
pos in time series: 511 neg in time series: 2715 pos-neg ratio: 0.18821362799263353 total: 3226


In [53]:
importlib.reload(ut)
ut.save_data(train_data, multipath, domain[select_domain], "train_data", times)
ut.save_data(node_feature, multipath, domain[select_domain], "node_feature", times)
ut.save_data(edge_list, multipath, domain[select_domain], "edge_list", times)

../linkPrediction/multi_edges/apnea\apnea-train_data_2008-2015.pkl
../linkPrediction/multi_edges/apnea\apnea-node_feature_2008-2015.pkl
../linkPrediction/multi_edges/apnea\apnea-edge_list_2008-2015.pkl


In [54]:
importlib.reload(fs)
importlib.reload(cl)

edge_list = ut.load_data(multipath, domain[select_domain], "edge_list", times)
edge_set = set(edge_list)
edges = list(g_test[2015].edges())
non_edges =  list(nx.non_edges(g_test[2015]))
r_edges = [(b, a) for a, b in edges]
r_non_edges = [(b, a) for a, b in non_edges]
test_edges = []
label = []
train_ed=[]
ins = set(edges).intersection(edge_set)
train_ed.extend(ins)
label.extend([1]*len(ins))
ins = set(r_edges).intersection(edge_set)
train_ed.extend(ins)
label.extend([1]*len(ins))
ins = set(non_edges).intersection(edge_set)
train_ed.extend(ins)
label.extend([0]*len(ins))
ins = set(r_non_edges).intersection(edge_set)
train_ed.extend(ins)
label.extend([0]*len(ins))
train_data={}
train_data[2015] = pd.DataFrame({'row_name': train_ed, 'label': label})

print(len(train_ed))

g_train[2015]=g_test[2015]
time=[2014,2015,2016,1,1,20]
test_node, test_data = fs.dynamic_graph_feature_set(df, key_list, train_data, g_train[2014], g_train, g_train_static, time)

t=2015
g=g_train[t]
f_node = list(g.nodes)[0]
result = calculate_pathsim(g,f_node)
test_data[t]['path_sim'] = test_data[t].apply(lambda row: result[row['row_name'][0]][row['row_name'][1]],axis=1)
time=[2007,2008,2015,1,1,20]

../linkPrediction/multi_edges/apnea
3226


In [55]:
importlib.reload(ut)
times=[2007,2008,2015,1,1,20]
ut.save_data(test_data, multipath, domain[select_domain], "test_data", times)
ut.save_data(test_node, multipath, domain[select_domain], "test_node", times)

../linkPrediction/multi_edges/apnea\apnea-test_data_2008-2015.pkl
../linkPrediction/multi_edges/apnea\apnea-test_node_2008-2015.pkl


In [56]:
importlib.reload(ut)
times=[2007,2008,2015,1,1,20]
train_data = ut.load_data(multipath, domain[select_domain], "train_data", times)
node_feature = ut.load_data(multipath, domain[select_domain], "node_feature", times)
edge_list = ut.load_data(multipath, domain[select_domain], "edge_list", times)
test_data = ut.load_data(multipath, domain[select_domain], "test_data", times)
test_node = ut.load_data(multipath, domain[select_domain], "test_node", times)
print('train data length:',len(train_data))
print('node feature length:',len(node_feature))
print('edge_list length:',len(edge_list))
print('test_data length:',len(test_data[2015]))
print('test_node length:',len(test_node[2015]))

../linkPrediction/multi_edges/apnea
../linkPrediction/multi_edges/apnea
../linkPrediction/multi_edges/apnea
../linkPrediction/multi_edges/apnea
../linkPrediction/multi_edges/apnea
train data length: 7
node feature length: 7
edge_list length: 3226
test_data length: 3226
test_node length: 814


In [57]:
#filter based on pathsim
edge_list=set()
for t in range(2008,2015):
    dd=train_data[t]
    ddn = dd[dd['path_sim']==0]
    ddp = dd[dd['path_sim']>0]
    ddl_n=ddn[ddn['label']==0]
    ddl_p=ddn[ddn['label']==1]
    p_len = len(dd[dd['path_sim']>0])
    mid=int(p_len/2)
    ddl_n = ddl_n.sample(n=mid, random_state=1)
    ddl_p = ddl_p.sample(n=p_len-mid, random_state=1)
    print(p_len,len(ddl_n),len(ddl_p))
    dd = pd.concat([ddl_n,ddl_p,ddp])
    dd=dd.sample(frac=1)
    train_data[t]=dd
    print(len(train_data[t]))
    edge_list.update(train_data[t]['row_name'])
edge_list = list(edge_list)

7 3 4
14
11 5 6
22
12 6 6
24
13 6 7
26
10 5 5
20
20 10 10
40
25 12 13
50


In [58]:
def reshape(train_data, node_feature, edge_list, test_data,test_node,times,features):
    cat ={25:0,5:1,3:2,1:3}
    ts = times[1]
    te = times[2]+1
    it_index = times[4]
    times_range = te - ts
    total_sample = len(edge_list)
    train_data[times[2]] = test_data
    node_feature[times[2]] = test_node
    feature_length = len(features['edge'])+32
    X = np.zeros([total_sample, times_range, feature_length])
    y = np.zeros(total_sample)
    print("X shape:", X.shape, "y shape:", y.shape)
    for id, edge in enumerate(edge_list):
        for t in range(ts, te, it_index):
            t_data = train_data[t]
            n_data = node_feature[t]
            node_list = set(n_data['node_index'])
            edge_list_t = set(t_data['row_name'])
            if edge in edge_list_t:
                train_row = np.asarray(t_data.loc[t_data['row_name'] == edge,features['edge']].values[0])
                X[id][t - ts][0:6] = train_row
                X[id][t - ts][3] = X[id][t - ts][3]*(t-ts+1)*2
            if edge[0] in node_list:
                node_row0 = np.asarray(n_data.loc[n_data['node_index'] == edge[0],features['node']].values[0])
                X[id][t - ts][6:9] = node_row0[0:3]
                X[id][t - ts][cat[node_row0[3]]+9] = 1
                X[id][t - ts][cat[node_row0[4]]+13] = 1
                X[id][t - ts][cat[node_row0[5]]+17] = 1
                X[id][t - ts][21] = node_row0[6]
                if edge[1] not in node_list:
                    X[id][t - ts][3] = node_row0[6]*(t-ts+1)
            if edge[1] in node_list:
                node_row1 = np.asarray(n_data.loc[n_data['node_index'] == edge[1],features['node']].values[0])
                X[id][t - ts][22:25] = node_row1[0:3]
                X[id][t - ts][cat[node_row1[3]]+25] = 1
                X[id][t - ts][cat[node_row1[4]]+29] = 1
                X[id][t - ts][cat[node_row1[5]]+33] = 1
                X[id][t - ts][37] = node_row1[6]
                if edge[0] not in node_list:
                    X[id][t - ts][3] = node_row1[6]*(t-ts+1)
        y[id] = test_data.loc[test_data['row_name'] == edge,'label'].values[0]
    return X, y

In [59]:
#####################  6           7        8       9,10,11,12     13,14,15,16      17,18,19,20      21
#####################  22          23       24      25,26,27,28    29,30,31,32      33,34,35,36      37
feature = {'node':['term_aut','term_art','degree','node_type_aut','node_type_art','node_type_deg','citation'],
           'edge':['typeaut','typeart','typenode','citation1','pref','path_sim']}
################       0         1         2           3        4      5

start = tm.time()
X, y = reshape(train_data, node_feature, edge_list, test_data[2015],test_node[2015],times,feature)
end = tm.time()
print(end - start)
ut.save_data(X, multipath, domain[select_domain], "X-features", times)
ut.save_data(y, multipath, domain[select_domain], "y-features", times)

X shape: (186, 8, 38) y shape: (186,)
4.631035566329956
../linkPrediction/multi_edges/apnea\apnea-X-features_2008-2015.pkl
../linkPrediction/multi_edges/apnea\apnea-y-features_2008-2015.pkl


In [60]:
import keras
import h5py

class CustomSaver(keras.callbacks.Callback):
    
    def __init__(self, param):
        self.name = param[3]
        self.times = param[4]
    def on_epoch_end(self, epoch, logs={}):
        ep_set =set([100,500,1000])
        if epoch in ep_set:  # or save after some epoch, each k-th epoch etc.
            ut.save_data(self.model, multipath, domain[select_domain], "model-"+name+"-"+str(epoch), self.times)
#             filename = domain + "-" + name + "_" + str(time[1]) + "-" + str(time[2]) + ".pkl"
#             filename_path = os.path.join(data_path, filename)
#             self.model.save('mmm.h5')

In [61]:
def create_linear(param):
        inputx = Input(shape=(param[0], param[1]))
        x = LSTM(20)(inputx)
        x = LSTM(20, activation="relu")(inputx)
        x = Dense(40, activation="relu")(x)
        x = Dense(20, activation="relu")(x)
        x = Dense(param[2],activation='linear',name="lin")(x)
        model = Model(inputs=inputx, outputs=x)
        return model

def create_cat1(param):
        inputx = Input(shape=(param[0], param[1]))
        x = LSTM(20)(inputx)
        x = Dense(20)(x)
        x = Dense(10,activation='relu')(x)
        x = Dense(param[2],activation='softmax',name="cat1")(x)
        model = Model(inputs=inputx, outputs=x)
        return model
    
def create_cat2(param):
        inputx = Input(shape=(param[0], param[1]))
        x = LSTM(20)(inputx)
        x = Dense(20)(x)
        x = Dense(10,activation='relu')(x)
        x = Dense(param[2],activation='softmax',name="cat2")(x)
        model = Model(inputs=inputx, outputs=x)
        return model
    
def lstm_forecast(X,param):
        X = ut.scale(X,0,1)
        y = X[:,7]
        X = X[:,0:7]
        batch_size = param[1]
        epoch = param[2]
        names1 = set(['author','article','degree'])
        names2 = set(['citation','pref','cm'])
        print(X.shape,y.shape)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = param[0], random_state = 0)
        ######## the first branch operates on the linear input

        lin_parameters = [X_train[:,:,0:3].shape[1], X_train[:,:,0:3].shape[2], y_train[:,0:3].shape[1]]
        lin = create_linear(lin_parameters)        
        
        saver = CustomSaver(param)
        
        if param[3] in names1:
            cat_parameters1 = [X_train[:,:,3:7].shape[1], X_train[:,:,3:7].shape[2], y_train[:,3:7].shape[1]]
            cat1 = create_cat1(cat_parameters1)

            cat_parameters2 = [X_train[:,:,7:11].shape[1], X_train[:,:,7:11].shape[2], y_train[:,7:11].shape[1]]
            cat2 = create_cat2(cat_parameters2)
        
            model = Model(inputs=[lin.input, cat1.input, cat2.input], outputs=[lin.output, cat1.output,cat2.output])
            model.compile(loss={'lin':'mse','cat1':'categorical_crossentropy','cat2':'categorical_crossentropy'},
                              optimizer='Adam',
                              metrics={'lin':'accuracy','cat1':'categorical_accuracy','cat2':'categorical_accuracy'})
            history = model.fit([X_train[:,:,0:3],X_train[:,:,3:7],X_train[:,:,7:11]],
                                [y_train[:,0:3],y_train[:,3:7],y_train[:,7:11]],
                                callbacks=[saver],
                                batch_size=batch_size,
                                epochs=epoch,
                                verbose=1)
        else:
            model = Model(inputs=lin.input, outputs=lin.output)
            model.compile(loss={'lin':'mse'},
                              optimizer='Adam',
                              metrics={'lin':'accuracy'})
            history = model.fit(X_train[:,:,0:3],
                                y_train[:,0:3],
                                callbacks=[saver],
                                batch_size=batch_size,
                                epochs=epoch,
                                verbose=1)
#         y_pr = model.predict([X_test[:,:,0:3],X_test[:,:,3:7],X_test[:,:,7:11]])
        return X_test, y_test

In [62]:
feature_names = {
'author':[0,6,22,9,10,11,12,25,26,27,28],
'article': [1,7,23,13,14,15,16,29,30,31,32],
'degree': [2,8,24,17,18,19,20,33,34,35,36],
'citation': [3,21,37],
'path_sim': [5,8,24]
}
names1 = set(['author','article','degree'])
names2 = set(['citation','pref','path_sim'])
# names2 = set(['path_sim'])
X = ut.load_data(multipath, domain[select_domain], "X-features", times)
y = ut.load_data(multipath, domain[select_domain], "y-features", times)
for name,feature in feature_names.items():
    if name in names2:
        param = [0.3,64,1001,name,times]
        print(name,"---------------------------------------------------------------------------------")
        X_test, y_test = lstm_forecast(X[:,:,feature],param)
        ut.save_data(X_test, multipath, domain[select_domain], "X_test-"+name, times)
        ut.save_data(y_test, multipath, domain[select_domain], "y_test-"+name, times)
    if name in names1:
        param = [0.3,64,1001,name,times]
        print(name,"---------------------------------------------------------------------------------")
        X_test, y_test = lstm_forecast(X[:,:,feature],param)
        ut.save_data(X_test, multipath, domain[select_domain], "X_test-"+name, times)
        ut.save_data(y_test, multipath, domain[select_domain], "y_test-"+name, times)

../linkPrediction/multi_edges/apnea
../linkPrediction/multi_edges/apnea
author ---------------------------------------------------------------------------------
(186, 7, 11) (186, 11)
Epoch 1/1001
130/130 [==============================] - 0s 392us/step - loss: 3.0333 - lin_loss: 0.3098 - cat1_loss: 1.3505 - cat2_loss: 1.3730 - lin_acc: 0.0385 - cat1_categorical_accuracy: 0.4692 - cat2_categorical_accuracy: 0.0923
Epoch 2/1001
130/130 [==============================] - 0s 362us/step - loss: 2.9411 - lin_loss: 0.2830 - cat1_loss: 1.3272 - cat2_loss: 1.3309 - lin_acc: 0.0385 - cat1_categorical_accuracy: 0.7385 - cat2_categorical_accuracy: 0.2308
Epoch 3/1001
130/130 [==============================] - 0s 385us/step - loss: 2.8665 - lin_loss: 0.2596 - cat1_loss: 1.3048 - cat2_loss: 1.3021 - lin_acc: 0.1923 - cat1_categorical_accuracy: 0.7538 - cat2_categorical_accuracy: 0.6462
Epoch 4/1001
130/130 [==============================] - 0s 423us/step - loss: 2.7994 - lin_loss: 0.2395 - cat1_los

130/130 [==============================] - 0s 324us/step - loss: 1.1075 - lin_loss: 0.0312 - cat1_loss: 0.6200 - cat2_loss: 0.4563 - lin_acc: 0.5231 - cat1_categorical_accuracy: 0.8000 - cat2_categorical_accuracy: 0.8923
Epoch 36/1001
130/130 [==============================] - 0s 331us/step - loss: 1.1001 - lin_loss: 0.0309 - cat1_loss: 0.6179 - cat2_loss: 0.4513 - lin_acc: 0.5231 - cat1_categorical_accuracy: 0.8000 - cat2_categorical_accuracy: 0.8923
Epoch 37/1001
130/130 [==============================] - 0s 269us/step - loss: 1.0905 - lin_loss: 0.0303 - cat1_loss: 0.6154 - cat2_loss: 0.4447 - lin_acc: 0.5231 - cat1_categorical_accuracy: 0.8000 - cat2_categorical_accuracy: 0.8923
Epoch 38/1001
130/130 [==============================] - 0s 338us/step - loss: 1.0844 - lin_loss: 0.0307 - cat1_loss: 0.6117 - cat2_loss: 0.4420 - lin_acc: 0.5231 - cat1_categorical_accuracy: 0.8000 - cat2_categorical_accuracy: 0.8923
Epoch 39/1001
130/130 [==============================] - 0s 292us/step - l

130/130 [==============================] - 0s 323us/step - loss: 1.0193 - lin_loss: 0.0226 - cat1_loss: 0.5953 - cat2_loss: 0.4014 - lin_acc: 0.6538 - cat1_categorical_accuracy: 0.8385 - cat2_categorical_accuracy: 0.9077
Epoch 70/1001
130/130 [==============================] - 0s 369us/step - loss: 1.0127 - lin_loss: 0.0239 - cat1_loss: 0.5881 - cat2_loss: 0.4007 - lin_acc: 0.6615 - cat1_categorical_accuracy: 0.8385 - cat2_categorical_accuracy: 0.9077
Epoch 71/1001
130/130 [==============================] - 0s 324us/step - loss: 1.0016 - lin_loss: 0.0244 - cat1_loss: 0.5775 - cat2_loss: 0.3997 - lin_acc: 0.6769 - cat1_categorical_accuracy: 0.8462 - cat2_categorical_accuracy: 0.9077
Epoch 72/1001
130/130 [==============================] - 0s 308us/step - loss: 0.9917 - lin_loss: 0.0226 - cat1_loss: 0.5694 - cat2_loss: 0.3996 - lin_acc: 0.6615 - cat1_categorical_accuracy: 0.8462 - cat2_categorical_accuracy: 0.9077
Epoch 73/1001
130/130 [==============================] - 0s 346us/step - l

130/130 [==============================] - 0s 345us/step - loss: 0.9435 - lin_loss: 0.0184 - cat1_loss: 0.5420 - cat2_loss: 0.3832 - lin_acc: 0.6692 - cat1_categorical_accuracy: 0.8462 - cat2_categorical_accuracy: 0.9000
Epoch 104/1001
130/130 [==============================] - 0s 352us/step - loss: 0.9461 - lin_loss: 0.0197 - cat1_loss: 0.5408 - cat2_loss: 0.3856 - lin_acc: 0.6615 - cat1_categorical_accuracy: 0.8462 - cat2_categorical_accuracy: 0.9000
Epoch 105/1001
130/130 [==============================] - 0s 338us/step - loss: 0.9483 - lin_loss: 0.0192 - cat1_loss: 0.5377 - cat2_loss: 0.3915 - lin_acc: 0.6538 - cat1_categorical_accuracy: 0.8462 - cat2_categorical_accuracy: 0.9000
Epoch 106/1001
130/130 [==============================] - 0s 323us/step - loss: 0.9500 - lin_loss: 0.0182 - cat1_loss: 0.5361 - cat2_loss: 0.3958 - lin_acc: 0.6769 - cat1_categorical_accuracy: 0.8462 - cat2_categorical_accuracy: 0.9000
Epoch 107/1001
130/130 [==============================] - 0s 331us/step

130/130 [==============================] - 0s 369us/step - loss: 0.8991 - lin_loss: 0.0176 - cat1_loss: 0.5094 - cat2_loss: 0.3721 - lin_acc: 0.6846 - cat1_categorical_accuracy: 0.8462 - cat2_categorical_accuracy: 0.9000
Epoch 138/1001
130/130 [==============================] - 0s 485us/step - loss: 0.9012 - lin_loss: 0.0200 - cat1_loss: 0.5102 - cat2_loss: 0.3710 - lin_acc: 0.6769 - cat1_categorical_accuracy: 0.8385 - cat2_categorical_accuracy: 0.9000
Epoch 139/1001
130/130 [==============================] - 0s 624us/step - loss: 0.8982 - lin_loss: 0.0198 - cat1_loss: 0.5090 - cat2_loss: 0.3694 - lin_acc: 0.6615 - cat1_categorical_accuracy: 0.8000 - cat2_categorical_accuracy: 0.9000
Epoch 140/1001
130/130 [==============================] - 0s 531us/step - loss: 0.8923 - lin_loss: 0.0178 - cat1_loss: 0.5074 - cat2_loss: 0.3672 - lin_acc: 0.6846 - cat1_categorical_accuracy: 0.8000 - cat2_categorical_accuracy: 0.9000
Epoch 141/1001
130/130 [==============================] - 0s 434us/step

...

130/130 [==============================] - 0s 162us/step - loss: 0.0249 - acc: 0.5308
Epoch 616/1001
130/130 [==============================] - 0s 215us/step - loss: 0.0246 - acc: 0.5154
Epoch 617/1001
130/130 [==============================] - 0s 185us/step - loss: 0.0247 - acc: 0.5231
Epoch 618/1001
130/130 [==============================] - 0s 177us/step - loss: 0.0248 - acc: 0.5385
Epoch 619/1001
130/130 [==============================] - 0s 200us/step - loss: 0.0251 - acc: 0.5462
Epoch 620/1001
130/130 [==============================] - 0s 176us/step - loss: 0.0260 - acc: 0.5308
Epoch 621/1001
130/130 [==============================] - 0s 171us/step - loss: 0.0259 - acc: 0.5231
Epoch 622/1001
130/130 [==============================] - 0s 200us/step - loss: 0.0253 - acc: 0.5385
Epoch 623/1001
130/130 [==============================] - 0s 177us/step - loss: 0.0249 - acc: 0.5308
Epoch 624/1001
130/130 [==============================] - 0s 177us/step - loss: 0.0247 - acc: 0.5077
Epoch

130/130 [==============================] - 0s 185us/step - loss: 0.0254 - acc: 0.5154
Epoch 697/1001
130/130 [==============================] - 0s 160us/step - loss: 0.0236 - acc: 0.5308
Epoch 698/1001
130/130 [==============================] - 0s 154us/step - loss: 0.0245 - acc: 0.5154
Epoch 699/1001
130/130 [==============================] - 0s 198us/step - loss: 0.0256 - acc: 0.5154
Epoch 700/1001
130/130 [==============================] - 0s 175us/step - loss: 0.0247 - acc: 0.5154
Epoch 701/1001
130/130 [==============================] - 0s 258us/step - loss: 0.0237 - acc: 0.5462
Epoch 702/1001
130/130 [==============================] - 0s 113us/step - loss: 0.0239 - acc: 0.5308
Epoch 703/1001
130/130 [==============================] - 0s 146us/step - loss: 0.0242 - acc: 0.5154
Epoch 704/1001
130/130 [==============================] - 0s 177us/step - loss: 0.0241 - acc: 0.5154
Epoch 705/1001
130/130 [==============================] - 0s 178us/step - loss: 0.0238 - acc: 0.5154
Epoch

130/130 [==============================] - 0s 200us/step - loss: 0.0223 - acc: 0.5462
Epoch 778/1001
130/130 [==============================] - 0s 139us/step - loss: 0.0223 - acc: 0.5462
Epoch 779/1001
130/130 [==============================] - 0s 138us/step - loss: 0.0224 - acc: 0.5462
Epoch 780/1001
130/130 [==============================] - 0s 172us/step - loss: 0.0224 - acc: 0.5462
Epoch 781/1001
130/130 [==============================] - 0s 177us/step - loss: 0.0223 - acc: 0.5538
Epoch 782/1001
130/130 [==============================] - 0s 180us/step - loss: 0.0222 - acc: 0.5308
Epoch 783/1001
130/130 [==============================] - 0s 177us/step - loss: 0.0220 - acc: 0.5462
Epoch 784/1001
130/130 [==============================] - 0s 185us/step - loss: 0.0220 - acc: 0.5538
Epoch 785/1001
130/130 [==============================] - 0s 177us/step - loss: 0.0221 - acc: 0.5538
Epoch 786/1001
130/130 [==============================] - 0s 146us/step - loss: 0.0222 - acc: 0.5462
Epoch

130/130 [==============================] - 0s 177us/step - loss: 0.0223 - acc: 0.5846
Epoch 859/1001
130/130 [==============================] - 0s 177us/step - loss: 0.0211 - acc: 0.5769
Epoch 860/1001
130/130 [==============================] - 0s 200us/step - loss: 0.0208 - acc: 0.5769
Epoch 861/1001
130/130 [==============================] - 0s 169us/step - loss: 0.0210 - acc: 0.5846
Epoch 862/1001
130/130 [==============================] - 0s 203us/step - loss: 0.0209 - acc: 0.5692
Epoch 863/1001
130/130 [==============================] - 0s 162us/step - loss: 0.0217 - acc: 0.5385
Epoch 864/1001
130/130 [==============================] - 0s 192us/step - loss: 0.0226 - acc: 0.5154
Epoch 865/1001
130/130 [==============================] - 0s 161us/step - loss: 0.0224 - acc: 0.5154
Epoch 866/1001
130/130 [==============================] - 0s 214us/step - loss: 0.0214 - acc: 0.5231
Epoch 867/1001
130/130 [==============================] - 0s 169us/step - loss: 0.0210 - acc: 0.5462
Epoch

130/130 [==============================] - 0s 140us/step - loss: 0.0196 - acc: 0.5923
Epoch 940/1001
130/130 [==============================] - 0s 167us/step - loss: 0.0198 - acc: 0.5615
Epoch 941/1001
130/130 [==============================] - 0s 172us/step - loss: 0.0205 - acc: 0.5538
Epoch 942/1001
130/130 [==============================] - 0s 177us/step - loss: 0.0204 - acc: 0.5615
Epoch 943/1001
130/130 [==============================] - 0s 139us/step - loss: 0.0199 - acc: 0.5615
Epoch 944/1001
130/130 [==============================] - 0s 239us/step - loss: 0.0201 - acc: 0.5692
Epoch 945/1001
130/130 [==============================] - 0s 228us/step - loss: 0.0210 - acc: 0.5385
Epoch 946/1001
130/130 [==============================] - 0s 193us/step - loss: 0.0208 - acc: 0.5308
Epoch 947/1001
130/130 [==============================] - 0s 200us/step - loss: 0.0199 - acc: 0.5385
Epoch 948/1001
130/130 [==============================] - 0s 216us/step - loss: 0.0205 - acc: 0.5000
Epoch

In [63]:
def create_class(param):
    inputx = Input(shape=(param[1],))
    x = Dense(20, activation='relu')(inputx)
    x = Dense(10,activation='relu')(x)
    x = Dense(1,activation='sigmoid',name="class")(x)
    model = Model(inputs=inputx, outputs=x)
    return model

def lstm_classification(X,y,param):
    batch_size = param[1]
    epoch = param[2]
    print(X.shape,y.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = param[0], random_state = 0)
    class_param = [X.shape[0], X.shape[1], 1]
    print(class_param)
    
    cl = create_class(class_param)
    model = Model(inputs=cl.input, outputs=cl.output)
    model.compile(loss={'class':'binary_crossentropy'},
                      optimizer='Adam',
                      metrics={'class':'accuracy'})
    print(X_train.shape,y_train.shape)
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        shuffle=True,
                        batch_size=batch_size,
                        epochs=epoch,
                        verbose=1)
    return X_test, y_test, model

In [64]:
feature_names = {
'author':[0,6,22,9,10,11,12,25,26,27,28],
'article': [1,7,23,13,14,15,16,29,30,31,32],
'degree': [2,8,24,17,18,19,20,33,34,35,36],
'citation': [3,21,37],
'path_sim': [5,8,24]
}

results = []
names1 = set(['author','article','degree'])
names2 = set(['citation','pref','path_sim'])
X = ut.load_data(multipath, domain[select_domain], "X-features", times)
y = ut.load_data(multipath, domain[select_domain], "y-features", times)
for name,feature in feature_names.items():
    param = [0.3,64,500,name,times]
    m=ut.load_data(multipath, domain[select_domain], "model-"+name+"-"+str(1000), times)
    X1=X[:,:,feature]
    X1=X1[:,0:7]
    print(name, X1.shape, "--------------------------------------------------------------------")
    if name in names1:
        y_pr = m.predict([X1[:,:,0:3],X1[:,:,3:7],X1[:,:,7:11]])
#         row_maxes = y_pr[1].max(axis=1).reshape(-1, 1)
#         y_pr[1][:] = np.where(y_pr[1] == row_maxes, 1, 0)
#         row_maxes = y_pr[2].max(axis=1).reshape(-1, 1)
#         y_pr[2][:] = np.where(y_pr[2] == row_maxes, 1, 0)
        y_pr1 = np.concatenate((y_pr[0],y_pr[1],y_pr[2]),axis=1)
    if name in names2:
        y_pr1 = m.predict(X1)
    X_test, y_test, model = lstm_classification(y_pr1,y,param)
    results.append(cl.model_evaluate(model, X_test, y_test, param[1], name))
ut.save_data(results, multipath, domain[select_domain], "results", times)

../linkPrediction/multi_edges/apnea
../linkPrediction/multi_edges/apnea
../linkPrediction/multi_edges/apnea
author (186, 7, 11) --------------------------------------------------------------------
(186, 11) (186,)
[186, 11, 1]
(130, 11) (130,)
Train on 130 samples, validate on 56 samples
Epoch 1/500
130/130 [==============================] - 0s 54us/step - loss: 5.6272 - acc: 0.3923 - val_loss: 5.9208 - val_acc: 0.3571
Epoch 2/500
130/130 [==============================] - 0s 39us/step - loss: 5.5459 - acc: 0.3923 - val_loss: 5.7649 - val_acc: 0.3571
Epoch 3/500
130/130 [==============================] - 0s 30us/step - loss: 5.4522 - acc: 0.3923 - val_loss: 5.6051 - val_acc: 0.3571
Epoch 4/500
130/130 [==============================] - 0s 35us/step - loss: 5.3090 - acc: 0.3923 - val_loss: 5.4464 - val_acc: 0.3571
Epoch 5/500
130/130 [==============================] - 0s 31us/step - loss: 5.1797 - acc: 0.3923 - val_loss: 5.2880 - val_acc: 0.3571
Epoch 6/500
130/130 [====================

130/130 [==============================] - 0s 51us/step - loss: 0.8134 - acc: 0.6154 - val_loss: 0.6613 - val_acc: 0.6250
Epoch 60/500
130/130 [==============================] - 0s 38us/step - loss: 0.8336 - acc: 0.6077 - val_loss: 0.7160 - val_acc: 0.6071
Epoch 61/500
130/130 [==============================] - 0s 31us/step - loss: 0.8448 - acc: 0.6000 - val_loss: 0.7837 - val_acc: 0.5357
Epoch 62/500
130/130 [==============================] - 0s 31us/step - loss: 1.0416 - acc: 0.5538 - val_loss: 1.2424 - val_acc: 0.5179
Epoch 63/500
130/130 [==============================] - 0s 31us/step - loss: 1.6751 - acc: 0.5154 - val_loss: 0.8439 - val_acc: 0.5357
Epoch 64/500
130/130 [==============================] - 0s 31us/step - loss: 0.8884 - acc: 0.5769 - val_loss: 0.6287 - val_acc: 0.6964
Epoch 65/500
130/130 [==============================] - 0s 35us/step - loss: 0.7595 - acc: 0.5846 - val_loss: 0.6882 - val_acc: 0.6429
Epoch 66/500
130/130 [==============================] - 0s 31us/step

130/130 [==============================] - 0s 55us/step - loss: 0.6164 - acc: 0.6000 - val_loss: 0.6056 - val_acc: 0.6964
Epoch 120/500
130/130 [==============================] - 0s 44us/step - loss: 0.6217 - acc: 0.6000 - val_loss: 0.6046 - val_acc: 0.6786
Epoch 121/500
130/130 [==============================] - 0s 43us/step - loss: 0.6166 - acc: 0.6000 - val_loss: 0.5962 - val_acc: 0.6786
Epoch 122/500
130/130 [==============================] - 0s 54us/step - loss: 0.6128 - acc: 0.6231 - val_loss: 0.5974 - val_acc: 0.6786
Epoch 123/500
130/130 [==============================] - 0s 38us/step - loss: 0.6251 - acc: 0.6154 - val_loss: 0.6085 - val_acc: 0.6964
Epoch 124/500
130/130 [==============================] - 0s 54us/step - loss: 0.6156 - acc: 0.6077 - val_loss: 0.6022 - val_acc: 0.6964
Epoch 125/500
130/130 [==============================] - 0s 38us/step - loss: 0.6067 - acc: 0.6000 - val_loss: 0.6233 - val_acc: 0.6786
Epoch 126/500
130/130 [==============================] - 0s 31

130/130 [==============================] - 0s 62us/step - loss: 0.6754 - acc: 0.5769 - val_loss: 0.9728 - val_acc: 0.3393
Epoch 180/500
130/130 [==============================] - 0s 38us/step - loss: 1.1347 - acc: 0.5308 - val_loss: 0.5903 - val_acc: 0.7321
Epoch 181/500
130/130 [==============================] - 0s 62us/step - loss: 0.6611 - acc: 0.6385 - val_loss: 0.5872 - val_acc: 0.6786
Epoch 182/500
130/130 [==============================] - 0s 46us/step - loss: 0.8120 - acc: 0.6385 - val_loss: 0.5985 - val_acc: 0.6964
Epoch 183/500
130/130 [==============================] - 0s 31us/step - loss: 0.8797 - acc: 0.6462 - val_loss: 0.6075 - val_acc: 0.6964
Epoch 184/500
130/130 [==============================] - ETA: 0s - loss: 1.0274 - acc: 0.625 - 0s 31us/step - loss: 0.9153 - acc: 0.6538 - val_loss: 0.6112 - val_acc: 0.6964
Epoch 185/500
130/130 [==============================] - 0s 54us/step - loss: 0.9279 - acc: 0.6538 - val_loss: 0.6103 - val_acc: 0.6964
Epoch 186/500
130/130 [=

130/130 [==============================] - 0s 60us/step - loss: 0.6297 - acc: 0.6615 - val_loss: 0.5910 - val_acc: 0.6964
Epoch 240/500
130/130 [==============================] - 0s 38us/step - loss: 0.6211 - acc: 0.6692 - val_loss: 0.5962 - val_acc: 0.7143
Epoch 241/500
130/130 [==============================] - 0s 46us/step - loss: 0.6082 - acc: 0.6385 - val_loss: 0.5957 - val_acc: 0.7143
Epoch 242/500
130/130 [==============================] - 0s 62us/step - loss: 0.6194 - acc: 0.6538 - val_loss: 0.5888 - val_acc: 0.6964
Epoch 243/500
130/130 [==============================] - 0s 38us/step - loss: 0.6409 - acc: 0.6615 - val_loss: 0.5888 - val_acc: 0.6964
Epoch 244/500
130/130 [==============================] - 0s 34us/step - loss: 0.6297 - acc: 0.6615 - val_loss: 0.5943 - val_acc: 0.6964
Epoch 245/500
130/130 [==============================] - 0s 38us/step - loss: 0.6148 - acc: 0.6538 - val_loss: 0.6051 - val_acc: 0.7321
Epoch 246/500
130/130 [==============================] - 0s 77

...

130/130 [==============================] - 0s 93us/step - loss: 0.6054 - acc: 0.6846 - val_loss: 0.6303 - val_acc: 0.6250
Epoch 216/500
130/130 [==============================] - 0s 29us/step - loss: 0.6077 - acc: 0.6846 - val_loss: 0.6230 - val_acc: 0.6250
Epoch 217/500
130/130 [==============================] - 0s 31us/step - loss: 0.6034 - acc: 0.6846 - val_loss: 0.6189 - val_acc: 0.6429
Epoch 218/500
130/130 [==============================] - 0s 38us/step - loss: 0.6041 - acc: 0.6846 - val_loss: 0.6180 - val_acc: 0.6429
Epoch 219/500
130/130 [==============================] - 0s 39us/step - loss: 0.6051 - acc: 0.6769 - val_loss: 0.6260 - val_acc: 0.6250
Epoch 220/500
130/130 [==============================] - 0s 50us/step - loss: 0.6043 - acc: 0.6615 - val_loss: 0.6542 - val_acc: 0.5179
Epoch 221/500
130/130 [==============================] - 0s 34us/step - loss: 0.6236 - acc: 0.6308 - val_loss: 0.6696 - val_acc: 0.5000
Epoch 222/500
130/130 [==============================] - 0s 27

130/130 [==============================] - 0s 31us/step - loss: 0.5917 - acc: 0.6154 - val_loss: 0.6131 - val_acc: 0.6250
Epoch 276/500
130/130 [==============================] - 0s 31us/step - loss: 0.5927 - acc: 0.6154 - val_loss: 0.6066 - val_acc: 0.6250
Epoch 277/500
130/130 [==============================] - 0s 46us/step - loss: 0.6053 - acc: 0.6231 - val_loss: 0.6069 - val_acc: 0.6429
Epoch 278/500
130/130 [==============================] - 0s 54us/step - loss: 0.6262 - acc: 0.6462 - val_loss: 0.6079 - val_acc: 0.6607
Epoch 279/500
130/130 [==============================] - 0s 46us/step - loss: 0.6365 - acc: 0.6308 - val_loss: 0.6079 - val_acc: 0.6607
Epoch 280/500
130/130 [==============================] - 0s 31us/step - loss: 0.6410 - acc: 0.6231 - val_loss: 0.6068 - val_acc: 0.6607
Epoch 281/500
130/130 [==============================] - 0s 35us/step - loss: 0.6369 - acc: 0.6231 - val_loss: 0.6044 - val_acc: 0.6607
Epoch 282/500
130/130 [==============================] - 0s 76

130/130 [==============================] - 0s 42us/step - loss: 0.5802 - acc: 0.6846 - val_loss: 0.6187 - val_acc: 0.6071
Epoch 336/500
130/130 [==============================] - 0s 30us/step - loss: 0.5776 - acc: 0.6769 - val_loss: 0.6223 - val_acc: 0.6071
Epoch 337/500
130/130 [==============================] - 0s 63us/step - loss: 0.5786 - acc: 0.6692 - val_loss: 0.6236 - val_acc: 0.6250
Epoch 338/500
130/130 [==============================] - 0s 38us/step - loss: 0.5786 - acc: 0.6769 - val_loss: 0.6229 - val_acc: 0.6071
Epoch 339/500
130/130 [==============================] - 0s 46us/step - loss: 0.5793 - acc: 0.6615 - val_loss: 0.6222 - val_acc: 0.6071
Epoch 340/500
130/130 [==============================] - 0s 23us/step - loss: 0.5795 - acc: 0.6462 - val_loss: 0.6215 - val_acc: 0.6250
Epoch 341/500
130/130 [==============================] - 0s 38us/step - loss: 0.5799 - acc: 0.6385 - val_loss: 0.6215 - val_acc: 0.6250
Epoch 342/500
130/130 [==============================] - 0s 62

130/130 [==============================] - 0s 46us/step - loss: 0.5947 - acc: 0.6385 - val_loss: 0.6447 - val_acc: 0.5714
Epoch 396/500
130/130 [==============================] - 0s 38us/step - loss: 0.5911 - acc: 0.6308 - val_loss: 0.6386 - val_acc: 0.5893
Epoch 397/500
130/130 [==============================] - 0s 77us/step - loss: 0.5867 - acc: 0.6308 - val_loss: 0.6352 - val_acc: 0.5893
Epoch 398/500
130/130 [==============================] - 0s 31us/step - loss: 0.5906 - acc: 0.6538 - val_loss: 0.6347 - val_acc: 0.5893
Epoch 399/500
130/130 [==============================] - 0s 38us/step - loss: 0.5929 - acc: 0.6692 - val_loss: 0.6341 - val_acc: 0.5893
Epoch 400/500
130/130 [==============================] - 0s 46us/step - loss: 0.5930 - acc: 0.6769 - val_loss: 0.6328 - val_acc: 0.6071
Epoch 401/500
130/130 [==============================] - 0s 71us/step - loss: 0.5929 - acc: 0.6769 - val_loss: 0.6309 - val_acc: 0.6250
Epoch 402/500
130/130 [==============================] - 0s 38

130/130 [==============================] - 0s 38us/step - loss: 0.5699 - acc: 0.6769 - val_loss: 0.6261 - val_acc: 0.6429
Epoch 456/500
130/130 [==============================] - 0s 62us/step - loss: 0.5695 - acc: 0.6769 - val_loss: 0.6239 - val_acc: 0.6250
Epoch 457/500
130/130 [==============================] - 0s 31us/step - loss: 0.5696 - acc: 0.6538 - val_loss: 0.6216 - val_acc: 0.6429
Epoch 458/500
130/130 [==============================] - 0s 36us/step - loss: 0.5729 - acc: 0.6462 - val_loss: 0.6207 - val_acc: 0.6250
Epoch 459/500
130/130 [==============================] - 0s 44us/step - loss: 0.5754 - acc: 0.6846 - val_loss: 0.6204 - val_acc: 0.6250
Epoch 460/500
130/130 [==============================] - 0s 39us/step - loss: 0.5833 - acc: 0.6923 - val_loss: 0.6203 - val_acc: 0.6250
Epoch 461/500
130/130 [==============================] - 0s 77us/step - loss: 0.5869 - acc: 0.6923 - val_loss: 0.6200 - val_acc: 0.6250
Epoch 462/500
130/130 [==============================] - 0s 32

In [69]:
#obesity
feature_names = {
'author':[0,6,22,9,10,11,12,25,26,27,28],
'article': [1,7,23,13,14,15,16,29,30,31,32],
'degree': [2,8,24,17,18,19,20,33,34,35,36],
'citation': [3,21,37],
'path_sim': [5,8,24]
}
names1 = set(['author','article','degree'])
names2 = set(['citation','pref','path_sim'])
epochs = [100,500,1000]
for name,feature in feature_names.items():
    X_test = ut.load_data(multipath, domain[select_domain], "X_test-"+name, times)
    y_test = ut.load_data(multipath, domain[select_domain], "y_test-"+name, times)
    for ep in epochs:      
        if name in names1:
            m=ut.load_data(multipath, domain[select_domain], "model-"+name+"-"+str(ep), times)    
            y_pr = m.predict([X_test[:,:,0:3],X_test[:,:,3:7],X_test[:,:,7:11]])
            mse = mean_squared_error(y_test[:,0],y_pr[0][:,0])
            rmse1 = sqrt(mse)
            mse = mean_squared_error(y_test[:,1],y_pr[0][:,1])
            rmse2 = sqrt(mse)
            print(name,"-",ep,":",rmse1," --- ",rmse2)
        if name in names2:
            m=ut.load_data(multipath, domain[select_domain], "model-"+name+"-"+str(ep), times)
            y_pr = m.predict(X_test)
            mse = mean_squared_error(y_test[:,0],y_pr[:,0])
            rmse1 = sqrt(mse)
            mse = mean_squared_error(y_test[:,1],y_pr[:,1])
            rmse2 = sqrt(mse)
            print(name,"-",ep,":",rmse1," --- ",rmse2)

../linkPrediction/multi_edges/obesity
../linkPrediction/multi_edges/obesity
../linkPrediction/multi_edges/obesity
author - 100 : 0.04164499403314429  ---  0.04880915772046294
../linkPrediction/multi_edges/obesity
author - 500 : 0.017591281764953337  ---  0.04803145441481252
../linkPrediction/multi_edges/obesity
author - 1000 : 0.022211323969704808  ---  0.04832855908707136
../linkPrediction/multi_edges/obesity
../linkPrediction/multi_edges/obesity
../linkPrediction/multi_edges/obesity
article - 100 : 0.036385826400577234  ---  0.061596833009407034
../linkPrediction/multi_edges/obesity
article - 500 : 0.02057239881907729  ---  0.06633705249592331
../linkPrediction/multi_edges/obesity
article - 1000 : 0.01843893224466915  ---  0.06560081098753696
../linkPrediction/multi_edges/obesity
../linkPrediction/multi_edges/obesity
../linkPrediction/multi_edges/obesity
degree - 100 : 0.05254451179376976  ---  0.07148267461157368
../linkPrediction/multi_edges/obesity
degree - 500 : 0.052973333460456

In [17]:
#obesity
results = ut.load_data(multipath, domain[select_domain], "results", times)
for result in results:
    print(result['model name'], result['auc'], result['test accuracy'])

../linkPrediction/multi_edges/obesity
author 0.618433392859437 0.6091624908389239
article 0.6441152665184937 0.6241947027215295
degree 0.593594091927147 0.6113099490819698
citation 0.5836727177734667 0.6077308527639959
path_sim 0.5315948713336844 0.5962777389060111


In [67]:
#apnea
feature_names = {
'author':[0,6,22,9,10,11,12,25,26,27,28],
'article': [1,7,23,13,14,15,16,29,30,31,32],
'degree': [2,8,24,17,18,19,20,33,34,35,36],
'citation': [3,21,37],
'path_sim': [5,8,24]
}
names1 = set(['author','article','degree'])
names2 = set(['citation','pref','path_sim'])
epochs = [100,500,1000]
for name,feature in feature_names.items():
    X_test = ut.load_data(multipath, domain[select_domain], "X_test-"+name, times)
    y_test = ut.load_data(multipath, domain[select_domain], "y_test-"+name, times)
    for ep in epochs:      
        if name in names1:
            m=ut.load_data(multipath, domain[select_domain], "model-"+name+"-"+str(ep), times)    
            y_pr = m.predict([X_test[:,:,0:3],X_test[:,:,3:7],X_test[:,:,7:11]])
            mse = mean_squared_error(y_test[:,0],y_pr[0][:,0])
            rmse1 = sqrt(mse)
            mse = mean_squared_error(y_test[:,1],y_pr[0][:,1])
            rmse2 = sqrt(mse)
            print(name,"-",ep,":",rmse1," --- ",rmse2)
        if name in names2:
            m=ut.load_data(multipath, domain[select_domain], "model-"+name+"-"+str(ep), times)
            y_pr = m.predict(X_test)
            mse = mean_squared_error(y_test[:,0],y_pr[:,0])
            rmse1 = sqrt(mse)
            mse = mean_squared_error(y_test[:,1],y_pr[:,1])
            rmse2 = sqrt(mse)
            print(name,"-",ep,":",rmse1," --- ",rmse2)

../linkPrediction/multi_edges/apnea
../linkPrediction/multi_edges/apnea
../linkPrediction/multi_edges/apnea
author - 100 : 0.11479666051128778  ---  0.10733451350963201
../linkPrediction/multi_edges/apnea
author - 500 : 0.07995998412427634  ---  0.10007941038988453
../linkPrediction/multi_edges/apnea
author - 1000 : 0.08491481210843109  ---  0.10136931271279201
../linkPrediction/multi_edges/apnea
../linkPrediction/multi_edges/apnea
../linkPrediction/multi_edges/apnea
article - 100 : 0.12293102263828799  ---  0.12668913433903806
../linkPrediction/multi_edges/apnea
article - 500 : 0.0849281577064248  ---  0.12527365535160762
../linkPrediction/multi_edges/apnea
article - 1000 : 0.07856671981952419  ---  0.12508754258390137
../linkPrediction/multi_edges/apnea
../linkPrediction/multi_edges/apnea
../linkPrediction/multi_edges/apnea
degree - 100 : 0.14163920370624572  ---  0.20982060754530651
../linkPrediction/multi_edges/apnea
degree - 500 : 0.144163397887472  ---  0.21079824043777154
../lin

In [66]:
#apnea
results = ut.load_data(multipath, domain[select_domain], "results", times)
for result in results:
    print(result['model name'], result['auc'], result['test accuracy'])

../linkPrediction/multi_edges/apnea
author 0.6125 0.5892857313156128
article 0.6722222222222223 0.6785714030265808
degree 0.6125 0.6071428656578064
citation 0.4527777777777778 0.375
path_sim 0.6444444444444445 0.6428571343421936
